In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#choose data import checkpoint for your purpose

In [ ]:
#import raw data for chromosome of choice - vcf, gtf, CDS regions from gtf, unique gene IDs from gtf,
#gene IDs that contain CDS regions (coding transcripts), UPIMAPI annotation file
pd.set_option('display.max_columns', None)
df_chr1_vcf = pd.read_csv("./chr_vcf/chr1.gtf", delimiter="\t", header=None)
df_chr1_gtf = pd.read_csv("./chr_gtf/chr1.gtf", delimiter="\t", header=None)
df_chr1_gtf_CDS = pd.read_csv("./chr_gtf/CDSs/chr1.gtf", delimiter="\t", header=None)
df_chr1_names = pd.read_csv("./chr_gtf/uniq_CDS1.txt", delimiter="\t", header=None)
df_prot_coding_names = pd.read_csv("./coding_transcripts.txt", delimiter="\t", header=None)
upimapi_df = pd.read_csv("/home/Maxim/Bioinformatics/General_software/UPIMAPI/output/CambridgeChrs_adjusted/CambridgeChrs_adjusted/merged/Cambridge_UPIMAPI.tsv", delimiter="\t")

#hard coded chromosomal length
chromosome_length = 68399915

In [ ]:
#import intermediates
SV_hits_test = pd.read_csv("./result/chr1.csv")
df_conserved = pd.read_csv("./conserved/conserved_transcripts_chr1.csv")
final1 = pd.read_csv("./result/processed/chr1.tsv")
SV_counts_df4 = pd.read_csv("./result/processed/counts/chr1.tsv")

overlap_df1 = pd.read_csv("./result/checkpoint_fin/overlaps/overlap_df_chr1.csv")
overlap_df1_CDS = pd.read_csv("./result/checkpoint_fin/overlaps/CDS/overlap_df_chr1.csv")
overlap_df1["Transcripts"] = overlap_df1["Transcripts"].fillna("[]").apply(lambda x: eval(x))
overlap_df1["Coords"] = overlap_df1["Coords"].fillna("[]").apply(lambda x: eval(x))
overlap_df1_CDS["Transcripts"] = overlap_df1_CDS["Transcripts"].fillna("[]").apply(lambda x: eval(x))
overlap_df1_CDS["Coords"] = overlap_df1_CDS["Coords"].fillna("[]").apply(lambda x: eval(x))

In [ ]:
#import all necessary data for plotting chromosome browser images
chromosome_length = 68399915
all_SV_coords = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/allSVchr1")
SV_coords = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/overlapSVchr1")
tr_cov_df = pd.read_csv("./result/checkpoint_fin/overlaps/tr_cov_plot/tr_cov_df_chr1.csv")
overlap_norm_SV_coords2 = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/NormSVLenPerTrChr1")
tr_cov_df_CDS = pd.read_csv("./result/checkpoint_fin/overlaps/CDS/tr_cov_plot/tr_cov_df_chr1.csv")
CDS_plot_SV_coords2 = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/CDSSVchr1")
CDS_overlaps_test = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/CDS_SV_densitychr1")
CDS_norm_SV_coords2 = pd.read_csv("./plotting/transcript_SV_coverage/checkpoint3/CDS_SV_densitychr1_coords")

In [ ]:
#fix raw data gene structures
df_chr1_gtf.columns=['Chr', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attribute']
df_chr1_gtf.insert(0, "TrName", df_chr1_names)

df_chr1_gtf_CDS.columns=['Chr', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attribute']
df_chr1_gtf_CDS.insert(0, "TrName", df_chr1_names)

df_chr1_vcf.columns=['#CHROM', "POS", "ID", "REF", "ALT", "QUAL", "FILTERS", "INFO", "FORMAT", "UnnamedSample"]

qseqid_df = upimapi_df["qseqid"].copy()
upimapi_df.insert(0, "TrName", qseqid_df.str[:-2])
test_df_upimapi = upimapi_df.astype(str).groupby("TrName")[['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen',
       'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'Entry',
       'Entry Name', 'Gene Names', 'Protein names', 'EC number',
       'Function [CC]', 'Pathway', 'Keywords', 'Protein existence',
       'Gene Ontology (GO)', 'Protein families', 'Taxonomic lineage',
       'Taxonomic lineage (Ids)', 'Organism (ID)', 'BioCyc', 'BRENDA', 'CDD',
       'eggNOG', 'Ensembl', 'InterPro', 'KEGG', 'Pfam', 'Reactome', 'RefSeq',
       'UniPathway', 'Taxonomic lineage (SUPERKINGDOM)',
       'Taxonomic lineage (PHYLUM)', 'Taxonomic lineage (CLASS)',
       'Taxonomic lineage (ORDER)', 'Taxonomic lineage (FAMILY)',
       'Taxonomic lineage (GENUS)', 'Taxonomic lineage (SPECIES)']].agg('! '.join).apply(lambda x: x[:].str.split('!')).reset_index()

In [ ]:
#filter out PASS, <30 QUAL, and non-homozygous from vcf
print(df_chr1_vcf.value_counts(subset="FILTERS"))
df_chr1_vcf = df_chr1_vcf[df_chr1_vcf["FILTERS"]=="PASS"]
df_chr1_vcf = df_chr1_vcf[df_chr1_vcf["QUAL"]>30]
df_chr1_vcf = df_chr1_vcf[df_chr1_vcf["UnnamedSample"].str.contains("1/1:")]
print(df_chr1_vcf.shape)

In [ ]:
#merge gtf and vcf by coordinate overlaps
SV_hits_test = pd.DataFrame()
counter = 0 
checkpoint = int(df_chr1_gtf.shape[0]/10)
for index_gtf, row_gtf in df_chr1_gtf.iterrows():
    counter += 1
    if counter % checkpoint == 0:
        SV_hits_test.to_csv("./result/chr1.csv")
        print("checkpoint: ", counter, "/ ", df_chr1_gtf.shape[0])
    vcf_df = df_chr1_vcf[(df_chr1_vcf["POS"] >= row_gtf[4]) & (df_chr1_vcf["POS"]<= row_gtf[5])]
    if vcf_df.shape[0] > 0:
        df_append = pd.concat([pd.DataFrame(row_gtf).transpose().reset_index(drop=True)]*vcf_df.shape[0])
        combined = pd.concat([df_append.reset_index(drop=True), vcf_df.reset_index(drop=True)], axis = 1)
        SV_hits_test = pd.concat([SV_hits_test, combined])
SV_hits_test.columns = ['TrName', 'Chr', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attribute', '#CHROM', "POS", "ID", "REF", "ALT", "QUAL", "FILTERS", "INFO", "FORMAT", "UnnamedSample"]
SV_hits_test.to_csv("./result/chr1.csv")

In [ ]:
#calculate vcf/gtf coverage stats
chromosome_length = 68399915
non_coding_length = 68399915
for index_gtf, row_gtf in df_chr1_gtf.iterrows():
    non_coding_length = non_coding_length - (row_gtf[5]-row_gtf[4])
coding_length = chromosome_length - non_coding_length
coding_percentage = 100/chromosome_length*coding_length
SV_mapped_percentage = 100/df_chr1_vcf.shape[0]*SV_hits_test.shape[0]
print("Chromosome length: ", chromosome_length, "\nCoding length: ", coding_length, 
      "\nNon coding length: ", non_coding_length, 
      "\nCoding percentage: ", "{:.2f}".format(coding_percentage), "%",
     "\nPercentage SVs mapped to coding regions: ", "{:.2f}".format(SV_mapped_percentage), "%")

In [ ]:
#calculate SV properties
SV_hits_test.reset_index(drop=True, inplace=True)
SV_hits_test["REF_LEN"] = SV_hits_test["REF"].apply(len)
SV_hits_test["ALT_LEN"] = SV_hits_test["ALT"].apply(len)
SV_hits_test["MUT_LEN"] = SV_hits_test["ALT_LEN"]-SV_hits_test["REF_LEN"]
SV_hits_test["MUT_TYPE"] = ""
SV_hits_test["MUT_TYPE"] = np.where(SV_hits_test["REF_LEN"]==SV_hits_test["ALT_LEN"], "Substitution", SV_hits_test["MUT_TYPE"])
SV_hits_test["MUT_TYPE"] = np.where(SV_hits_test["REF_LEN"]>SV_hits_test["ALT_LEN"], "Deletion", SV_hits_test["MUT_TYPE"])
SV_hits_test["MUT_TYPE"] = np.where(SV_hits_test["REF_LEN"]<SV_hits_test["ALT_LEN"], "Insertion", SV_hits_test["MUT_TYPE"])
SV_hits_test["SUB_TYPE"] = ""
SV_hits_test["SUB_TYPE"] = np.where((SV_hits_test["REF"]=="A") & (SV_hits_test["ALT"]=="G") | (SV_hits_test["REF"]=="G") & (SV_hits_test["ALT"]=="A") | (SV_hits_test["REF"]=="C") & (SV_hits_test["ALT"]=="T") | (SV_hits_test["REF"]=="T") & (SV_hits_test["ALT"]=="C"), "Transition", SV_hits_test["SUB_TYPE"])
SV_hits_test["SUB_TYPE"] = np.where((SV_hits_test["REF"]=="A") & (SV_hits_test["ALT"]=="C") | (SV_hits_test["REF"]=="C") & (SV_hits_test["ALT"]=="A") | (SV_hits_test["REF"]=="G") & (SV_hits_test["ALT"]=="T") | (SV_hits_test["REF"]=="T") & (SV_hits_test["ALT"]=="G") | (SV_hits_test["REF"]=="T") & (SV_hits_test["ALT"]=="A") | (SV_hits_test["REF"]=="A") & (SV_hits_test["ALT"]=="T") | (SV_hits_test["REF"]=="C") & (SV_hits_test["ALT"]=="G") | (SV_hits_test["REF"]=="G") & (SV_hits_test["ALT"]=="C"), "Transversion", SV_hits_test["SUB_TYPE"])
SV_hits_test["SUB_TYPE"] = np.where((SV_hits_test["SUB_TYPE"] == "") & (SV_hits_test["MUT_TYPE"]=="Substitution"), "Multi-substitution", SV_hits_test["SUB_TYPE"])

In [ ]:
#display SV property statistics 
print("Insertions :", SV_hits_test["MUT_TYPE"].value_counts()["Insertion"], 
      "\nDeletions :", SV_hits_test["MUT_TYPE"].value_counts()["Deletion"], 
      "\nSubstitutions :", SV_hits_test["MUT_TYPE"].value_counts()["Substitution"], 
      "\nTransitions: ", SV_hits_test["SUB_TYPE"].value_counts()["Transition"], 
      "\nTransversions :", SV_hits_test["SUB_TYPE"].value_counts()["Transversion"], 
      "\nMulti-substitutions", 0)

#no Multi-substitution SVs are found after filtering the vcf, hence it's commented out
#SV_hits_test["SUB_TYPE"].value_counts()["Multi-substitution"])

In [ ]:
#Expand the main dataframe with upimapi data and label CDS-containing transcripts
final = SV_hits_test.merge(test_df_upimapi, on=["TrName"], how="left")
df_prot_coding_names["Coding"] = "Yes"
df_prot_coding_names.columns=["TrName","Coding"]
final1 = final.merge(df_prot_coding_names, on=["TrName"], how="left")
final1.loc[final1["Coding"]!="Yes", "Coding"] = "No"

In [ ]:
#Coding vs non-coding transcript stats
protein_coding_length = 0
for index_gtf, row_gtf in final1[final1["Coding"]=="Yes"].drop_duplicates(subset=["TrName"]).iterrows():
    protein_coding_length = protein_coding_length + (row_gtf[5]-row_gtf[4])
non_protein_coding_length = coding_length - protein_coding_length

print("Protein-coding length: ", protein_coding_length, "\nProtein vs non-protein %: ",
      "{:.2f}".format(100/coding_length*protein_coding_length), "%",
      "\n% SVs found in protein-coding transcripts: ", 
      "{:.2f}".format(100/final1.shape[0]*final1[final1["Coding"]=="Yes"].shape[0]), "%")

In [ ]:
#Find conserved/identical transcripts between CW and DP
df_all = df_chr1_gtf.merge(final1.drop_duplicates(subset=["TrName"]), on=["TrName"], how="left", indicator=True)
df_conserved = df_all[df_all["_merge"] == "left_only"]
df_conserved.to_csv("./conserved/conserved_transcripts_chr1.csv")
print("Number of conserved transcripts:", df_conserved.shape[0], "/", df_chr1_gtf.shape[0])

In [ ]:
#frequency of SVs per transcript
SV_counts_df = pd.DataFrame(final1["TrName"].value_counts())
SV_counts_df.reset_index(inplace=True)

In [ ]:
#length of SVs per transcript sorted by descending order
SV_transcript_len = final1.groupby("TrName")[['MUT_LEN']].sum().sort_values(by = ["MUT_LEN"], ascending = False ).reset_index()

In [ ]:
#length of SVs per transcript sorted by descending SV frequency
SV_counts_df1 = SV_counts_df.merge(SV_transcript_len, on=["TrName"], how="left")

In [ ]:
#absolute length of SVs per transcript sorted in descending order
SV_transcript_len_abs = final1.groupby("TrName")[['MUT_LEN']].apply(lambda c:c.abs().sum()).sort_values(by = ["MUT_LEN"], ascending = False ).reset_index()

In [ ]:
#absolute length of SVs per transcript sorted by SV frequency
SV_transcript_len_abs = final1.groupby("TrName")[['MUT_LEN']].apply(lambda c:c.abs().sum()).sort_values(by = ["MUT_LEN"], ascending = False ).reset_index()
SV_counts_df2 = SV_counts_df.merge(SV_transcript_len_abs, on=["TrName"], how="left")

In [ ]:
#SV length to gene length ratio
SV_transcript_len_abs = final1.groupby("TrName")[['MUT_LEN']].apply(lambda c:c.abs().sum()).sort_values(by = ["MUT_LEN"], ascending = False ).reset_index()

In [ ]:
#compute transcript length and normalise
SV_counts_df1["Index"] = SV_counts_df1.index

final1["TrLength"] = final1["End"] - final1["Start"]

SV_counts_df3 = SV_counts_df2.merge(final1.drop_duplicates(subset=["TrName"]), on=["TrName"], how = "left")

SV_counts_df3["#SV:MUT_LEN_abs"] = SV_counts_df3["count"]/SV_counts_df3["MUT_LEN_x"]
SV_counts_df3["#SV:TrLength"] = SV_counts_df3["count"]/SV_counts_df3["TrLength"]
SV_counts_df3_no_inf = SV_counts_df3.replace([np.inf, -np.inf], np.nan)
SV_counts_df3["Index"] = SV_counts_df3.index

SV_counts_df4 = SV_counts_df3.merge(SV_counts_df1[["Index","MUT_LEN"]], on="Index", how="left", )

SV_counts_df4["#MUT_LEN_total:TrLength"] = SV_counts_df4["MUT_LEN"]/SV_counts_df4["TrLength"]
SV_counts_df4["#MUT_LEN_abs:TrLength"] = SV_counts_df4["MUT_LEN_x"]/SV_counts_df4["TrLength"]

In [ ]:
#save checkpoints
final1.to_csv("./result/checkpoint_fin/processed/chr1.tsv")
SV_counts_df4.to_csv("./result/checkpoint_fin/processed/counts/chr1.tsv")

In [ ]:
#NOT INCLUDED IN THESIS - GO ANNOTATIONS

In [ ]:
#flat GO term histogram
plt.rcParams['font.size'] = '16'
GO_list = SV_counts_df4["Gene Ontology (GO)"].explode()
GO_list.dropna(inplace=True)
GO_list = GO_list[GO_list != "nan"]
GO_list = GO_list[GO_list != " nan"]
GO_list = GO_list + ";"
GO_list = GO_list.str.split(pat = ";")
GO_list = GO_list.explode()
GO_list = GO_list[GO_list != ""]
vcounts = pd.DataFrame(GO_list.value_counts())
vcounts.index.name = "GO"
vcounts.reset_index(inplace=True)
vcounts["GO"] = vcounts["GO"].str.strip()
vcounts_df = vcounts.groupby(["GO"]).apply(lambda c:c.abs().sum()).sort_values(by = ["count"], ascending = False ).reset_index()
fig, axes = plt.subplots(figsize=(15,20))
vcounts_df.iloc[:50].plot(kind="barh", x="GO", y="count", ax=axes)

In [ ]:
#prepare GO sample list and save for processing using goatools
GO_list.index.name = "Index"
GO_list_df = GO_list.reset_index(inplace=False)
GO_list_df["Gene Ontology (GO)"] = GO_list_df["Gene Ontology (GO)"].str.strip()
GO_list_df1 = SV_counts_df4["TrName"]
GO_list_df1.index.name = "Index"
GO_list_df1 = GO_list_df1.reset_index(inplace=False)
GO_list_df2 = GO_list_df.merge(GO_list_df1, on=["Index"], how="left")
GO_list_df3 = GO_list_df2[["TrName", "Gene Ontology (GO)"]]

GO_list_df3["TrName"].to_csv("./GO/chr1_study.txt", index = False, header=False)

In [ ]:
#import goatools results - GO enrichment results - enriched gene groups in the chromosome (highly experimental)
import re

GO_enriched = pd.read_excel("./GO/chr_results/chr1.xlsx")

regex = []
for values in GO_enriched["ratio_in_pop"]:
    regex.append(re.search(r'\d+(?=\/)', values).group())
GO_enriched["pop_count"] = regex
GO_enriched["percent"] = GO_enriched.study_count.astype(int) / GO_enriched.pop_count.astype(int)
GO_enriched = GO_enriched.sort_values(by="p_fdr_bh").reset_index(drop=True)
GO_enriched["Index"] = GO_enriched.index

In [ ]:
#plot enrichments
import matplotlib as mpl
from matplotlib import cm
import seaborn as sns
import textwrap

fix, ax = plt.subplots(figsize=(0.5, 3))

cmap = mpl.cm.bwr_r
#norm = mpl.colors.PowerNorm(vmin = GO_enriched.p_fdr_bh.min(),  vmax = GO_enriched.p_fdr_bh.max(), gamma = 0.5)
norm = mpl.colors.Normalize(vmin = GO_enriched.p_fdr_bh.min(),  vmax = GO_enriched.p_fdr_bh.max())

mapper = cm.ScalarMappable(norm = norm, cmap = cm.bwr_r)

cbl = mpl.colorbar.ColorbarBase(ax, cmap = cmap, norm = norm, orientation = 'vertical')

plt.figure(figsize = (15,200))

ax = sns.barplot(data = GO_enriched, x = "percent", y = "name", palette = mapper.to_rgba(GO_enriched.p_fdr_bh.values), order = GO_enriched.sort_values("p_fdr_bh").name)

ax.set_yticklabels([textwrap.fill(e, 22) for e in GO_enriched["name"]])

plt.show()

In [ ]:
#Calculation of transcript overlap regions, CDS overlap regions, overlap region SV density

In [ ]:
#calculate overlaping transcript regions and prepare for plotting

df_chr1_gtf.sort_values(by='Start', inplace=True)

overlapping1 = []
overlapping_coverage1 = []
group_counter = 0
i = 0

while i < len(df_chr1_gtf):
    start = df_chr1_gtf.iloc[i]['Start']
    end = df_chr1_gtf.iloc[i]['End']
    overlap_group = [i]
    
    i += 1
    
    while i < len(df_chr1_gtf) and df_chr1_gtf.iloc[i]['Start'] <= end:
        end = max(end, df_chr1_gtf.iloc[i]['End'])
        overlap_group.append(i)
        i += 1
    
    overlapping1.append(overlap_group)
    overlapping_coverage1.append([start, end])

overlap_df1 = pd.DataFrame({
    'Transcripts': overlapping1,
    'Coords': overlapping_coverage1,
    'TrNum': range(len(overlapping1))
})

empty_regions = []
for i in range(1, len(overlapping_coverage1)):
    empty_regions.append([overlapping_coverage1[i - 1][1] + 1, overlapping_coverage1[i][0] - 1])

empty_regions.append([0, overlapping_coverage1[0][0] - 1])
empty_regions.append([overlapping_coverage1[-1][1] + 1, chromosome_length])

dfdf1 = pd.DataFrame(empty_regions, columns=['Left', 'Right'])
dfdf1['Coverage'] = 0
dfdf3 = pd.concat([dfdf1["Left"], dfdf1["Right"]], axis = 0)
dfdf3 = dfdf3.to_frame(name = "Coords")
dfdf3["Coverage"] = 0
overlap_df1["Left"] = [item[0] for item in overlap_df1["Coords"].values]
overlap_df1["Right"] = [item[1] for item in overlap_df1["Coords"].values]
dfdf4 = pd.concat([overlap_df1["Left"], overlap_df1["Right"]], axis = 0)  
dfdf4 = dfdf4.to_frame(name = "Coords")
dfdf4["Coverage"] = 1
tr_cov_df = pd.concat([dfdf3, dfdf4], axis = 0)

print("Done")

In [ ]:
#import CDS gtf
df_chr1_gtf_CDS = pd.read_csv("./chr_gtf/CDSs/chr1.gtf", delimiter="\t", header=None)
df_chr1_gtf_CDS.columns=['Chr', 'Source', 'Feature', 'Start', 'End', 'Score', 'Strand', 'Frame', 'Attribute']
df_chr1_gtf_CDS.insert(0, "TrName", df_chr1_names)

In [ ]:
#calculate overlaping CDS regions and prepare for plotting

df_chr1_gtf_CDS.sort_values(by='Start', inplace=True)

overlapping1 = []
overlapping_coverage1 = []
group_counter = 0
i = 0

while i < len(df_chr1_gtf_CDS):
    start = df_chr1_gtf_CDS.iloc[i]['Start']
    end = df_chr1_gtf_CDS.iloc[i]['End']
    overlap_group = [i]
    
    i += 1
    
    while i < len(df_chr1_gtf_CDS) and df_chr1_gtf_CDS.iloc[i]['Start'] <= end:
        end = max(end, df_chr1_gtf_CDS.iloc[i]['End'])
        overlap_group.append(i)
        i += 1
    
    overlapping1.append(overlap_group)
    overlapping_coverage1.append([start, end])

overlap_df1_CDS = pd.DataFrame({
    'Transcripts': overlapping1,
    'Coords': overlapping_coverage1,
    'TrNum': range(len(overlapping1))
})

empty_regions = []
for i in range(1, len(overlapping_coverage1)):
    empty_regions.append([overlapping_coverage1[i - 1][1] + 1, overlapping_coverage1[i][0] - 1])

empty_regions.append([0, overlapping_coverage1[0][0] - 1])
empty_regions.append([overlapping_coverage1[-1][1] + 1, chromosome_length])

dfdf1 = pd.DataFrame(empty_regions, columns=['Left', 'Right'])
dfdf1['Coverage'] = 0
dfdf3 = pd.concat([dfdf1["Left"], dfdf1["Right"]], axis = 0)
dfdf3 = dfdf3.to_frame(name = "Coords")
dfdf3["Coverage"] = 0
overlap_df1_CDS["Left"] = [item[0] for item in overlap_df1_CDS["Coords"].values]
overlap_df1_CDS["Right"] = [item[1] for item in overlap_df1_CDS["Coords"].values]
dfdf4 = pd.concat([overlap_df1_CDS["Left"], overlap_df1_CDS["Right"]], axis = 0)  
dfdf4 = dfdf4.to_frame(name = "Coords")
dfdf4["Coverage"] = 1
tr_cov_df_CDS = pd.concat([dfdf3, dfdf4], axis = 0)

print("Done")

In [ ]:
#SVs DF
pd.set_option('display.max_columns', None)
SVs_df = final1.drop_duplicates(subset=["POS"])
SVs_df["abs_MUT_LEN"] = SVs_df["MUT_LEN"].abs()
df_chr1_gtf["TrLen"] = df_chr1_gtf["End"] - df_chr1_gtf["Start"] 

#SV coverage plot data - transcript only
empty_regions = []
empty_regions.append(0)
empty_regions.append(chromosome_length)

SVs_df_re = SVs_df.reset_index()

for index, row in SVs_df_re.iterrows():
    empty_regions.append(SVs_df_re["POS"][index] -1)
    empty_regions.append(SVs_df_re["POS"][index] + 1)

empty_regions = list(set(empty_regions))

SV_coords = pd.DataFrame()
SV_coords["Coords"] = empty_regions
SV_coords["abs_MUT_LEN"] = 0
SV_coords1 = pd.DataFrame()
SV_coords1["Coords"] = SVs_df_re["POS"]
SV_coords1["abs_MUT_LEN"] = SVs_df_re["abs_MUT_LEN"]
SV_coords1["abs_MUT_LEN"].replace(0, 1, inplace=True)
SV_coords = SV_coords[SV_coords["Coords"].isin(SV_coords["Coords"])]
SV_coords = pd.concat([SV_coords, SV_coords1], axis = 0)

#SV coverage plot data - all SVs
all_SVs_df = df_chr1_vcf.drop_duplicates(subset=["POS"])
all_SVs_df["REF_LEN"] = all_SVs_df["REF"].apply(len)
all_SVs_df["ALT_LEN"] = all_SVs_df["ALT"].apply(len)
all_SVs_df["MUT_LEN"] = all_SVs_df["ALT_LEN"]-all_SVs_df["REF_LEN"]
all_SVs_df["abs_MUT_LEN"] = all_SVs_df["MUT_LEN"].abs()

empty_regions = []
empty_regions.append(0)
empty_regions.append(chromosome_length)

all_SVs_df_re = all_SVs_df.reset_index()

for index, row in all_SVs_df_re.iterrows():
    empty_regions.append(all_SVs_df_re["POS"][index] -1)
    empty_regions.append(all_SVs_df_re["POS"][index] + 1)

empty_regions = list(set(empty_regions))

all_SV_coords = pd.DataFrame()
all_SV_coords["Coords"] = empty_regions
all_SV_coords["abs_MUT_LEN"] = 0
all_SV_coords1 = pd.DataFrame()
all_SV_coords1["Coords"] = all_SVs_df_re["POS"]
all_SV_coords1["abs_MUT_LEN"] = all_SVs_df_re["abs_MUT_LEN"]
all_SV_coords1["abs_MUT_LEN"].replace(0, 1, inplace=True)
all_SV_coords = all_SV_coords[all_SV_coords["Coords"].isin(all_SV_coords["Coords"])]
all_SV_coords = pd.concat([all_SV_coords, all_SV_coords1], axis = 0)

In [ ]:
#overlap length and overlap midpoint
overlap_df1["Overlap_len"] =  overlap_df1["Right"] - overlap_df1["Left"]
overlap_df1["Midpoint_coord"] =  ((overlap_df1["Right"] + overlap_df1["Left"])/2).astype(int)

#index all protein-coding transcripts in context of all transcripts as ordered by starting coordinate
df_chr1_gtf_sorted = df_chr1_gtf.sort_values(by=["Start"])
df_chr1_gtf_sorted = df_chr1_gtf_sorted.reset_index(drop=True)
df_chr1_gtf_sorted["index1"] = df_chr1_gtf_sorted.index
all_transcripts_df = df_chr1_gtf_sorted[["TrName", "index1"]]
final1_overlaps = final1.merge(all_transcripts_df, on=["TrName"], how="left")

#calculate absolute mutation length of all SVs in protein coding transcripts
final1_overlaps = final1.merge(all_transcripts_df, on=["TrName"], how="left")
final1_overlaps = final1_overlaps.sort_values(by=["Start"])
final1_overlaps = final1_overlaps.drop_duplicates(subset=["POS"])
final1_overlaps["abs_MUT_LEN_diff"] = final1_overlaps["MUT_LEN"].abs()
final1_overlaps["abs_MUT_LEN"] = final1_overlaps["abs_MUT_LEN_diff"].abs() + 1
final1_overlaps["MUT_LEN_fixed"] = final1_overlaps[["REF_LEN", "ALT_LEN"]].max(axis=1)

#sum up absolute mutation lengths for each protein coding transcript
SV_overlaps_len_abs = final1_overlaps.groupby("TrName")[['MUT_LEN_fixed']].apply(lambda c:c.abs().sum()).reset_index()
final1_overlaps1 = final1_overlaps.drop_duplicates(subset=["index1"])
SV_overlaps_len_abs2 =  final1_overlaps1.merge(SV_overlaps_len_abs, on=["TrName"], how="left")

#merge protein coding transcript absolute mutation length data with transcript overlap coordinate data
index1_to_mut_len = SV_overlaps_len_abs2.groupby('index1')['MUT_LEN_fixed_y'].sum().to_dict()

def calculate_overlap_mut_len(row):
    return sum(index1_to_mut_len.get(index, 0) for index in row['Transcripts'])

overlap_df1['overlap_MUT_LEN'] = overlap_df1.apply(calculate_overlap_mut_len, axis=1)

    
#calculate normalised absolute mutation length per transcript (SV density)
overlap_df1["norm_overlap_MUT_LEN"] = overlap_df1["overlap_MUT_LEN"]/overlap_df1["Overlap_len"]

#calculate flanking coordinates for plotting
empty_regions = []
empty_regions.append(0)
empty_regions.append(chromosome_length)

overlap_df1 = overlap_df1.reset_index()

for index, row in overlap_df1.iterrows():
    empty_regions.append(overlap_df1["Midpoint_coord"][index] -1)
    empty_regions.append(overlap_df1["Midpoint_coord"][index] + 1)

empty_regions = list(set(empty_regions))

overlap_norm_SV_coords = pd.DataFrame()
overlap_norm_SV_coords["Coords"] = empty_regions
overlap_norm_SV_coords["norm_overlap_MUT_LEN"] = 0.0
overlap_norm_SV_coords1 = pd.DataFrame()
overlap_norm_SV_coords1["Coords"] = overlap_df1["Midpoint_coord"]
overlap_norm_SV_coords1["norm_overlap_MUT_LEN"] = overlap_df1["norm_overlap_MUT_LEN"]
overlap_norm_SV_coords2 = pd.concat([overlap_norm_SV_coords, overlap_norm_SV_coords1], axis = 0)

In [ ]:
#CDS SV plotting
SVs_for_CDSs = final1.drop_duplicates(subset=["POS"])

#need POS per CDS overlap and need SV density per CDS overlap
intervals = pd.IntervalIndex.from_arrays(overlap_df1_CDS["Left"], overlap_df1_CDS["Right"], "both")
SVs_in_CDSs = SVs_for_CDSs[intervals.get_indexer(SVs_for_CDSs.POS.values) != -1]
SVs_in_CDSs.drop_duplicates(subset="POS")

SVs_per_CDS = pd.DataFrame()
for index, row in overlap_df1_CDS.iterrows():
    interval = [pd.Interval(overlap_df1_CDS["Left"][index], overlap_df1_CDS["Right"][index], "both")]
    interval_index = pd.IntervalIndex(interval)
    SVs_in_CDS = SVs_in_CDSs[interval_index.get_indexer(SVs_in_CDSs.POS.values) != -1]
    SVs_in_CDS["CDS_start"] = overlap_df1_CDS["Left"][index]
    SVs_in_CDS["CDS_end"] = overlap_df1_CDS["Right"][index]
    SVs_per_CDS = pd.concat([SVs_per_CDS, SVs_in_CDS], axis = 0)
    
SVs_per_CDS["MUT_LEN_fixed"] = SVs_per_CDS[["REF_LEN", "ALT_LEN"]].max(axis=1)
SVs_per_CDS_abs = SVs_per_CDS.groupby("CDS_start")[['MUT_LEN_fixed']].apply(lambda c:c.abs().sum()).reset_index()
SVs_per_CDS_abs = SVs_per_CDS_abs.rename(columns={"MUT_LEN_fixed":"MUT_LEN_per_CDS"})
SVs_per_CDS =  SVs_per_CDS.merge(SVs_per_CDS_abs, on=["CDS_start"], how="left")
SVs_per_CDS["CDS_overlap_len"] =  SVs_per_CDS["CDS_end"] - SVs_per_CDS["CDS_start"]
SVs_per_CDS["CDS_midpoint_coord"] =  ((SVs_per_CDS["CDS_end"] + SVs_per_CDS["CDS_start"])/2).astype(int)
SVs_per_CDS["norm_overlap_MUT_LEN"] = SVs_per_CDS["MUT_LEN_per_CDS"]/SVs_per_CDS["CDS_overlap_len"]

#calculate flanking coordinates for plotting norm CDS SV density
CDS_overlaps = overlap_df1_CDS.copy()
CDS_overlaps = CDS_overlaps.rename(columns={"Left":"CDS_start","Right":"CDS_end"})
CDS_overlaps_test = CDS_overlaps.merge(SVs_per_CDS.drop_duplicates(subset=["CDS_start"]), on=["CDS_start"], how="left")

empty_regions = []
empty_regions.append(0)
empty_regions.append(chromosome_length)

CDS_overlaps_test1 = CDS_overlaps_test[["CDS_midpoint_coord", "norm_overlap_MUT_LEN"]].dropna().sort_values(by=["CDS_midpoint_coord"])
CDS_overlaps_test1 = CDS_overlaps_test1.reset_index()

for index, row in CDS_overlaps_test1.iterrows():
    empty_regions.append(CDS_overlaps_test1["CDS_midpoint_coord"][index] -1)
    empty_regions.append(CDS_overlaps_test1["CDS_midpoint_coord"][index] + 1)

empty_regions = list(set(empty_regions))

CDS_norm_SV_coords = pd.DataFrame()
CDS_norm_SV_coords["Coords"] = empty_regions
CDS_norm_SV_coords["norm_overlap_MUT_LEN"] = 0.0
CDS_norm_SV_coords1 = pd.DataFrame()
CDS_norm_SV_coords1["Coords"] = CDS_overlaps_test1["CDS_midpoint_coord"]
CDS_norm_SV_coords1["norm_overlap_MUT_LEN"] = CDS_overlaps_test1["norm_overlap_MUT_LEN"]
CDS_norm_SV_coords2 = pd.concat([CDS_norm_SV_coords, CDS_norm_SV_coords1], axis = 0)

CDS_SV_coords = SVs_per_CDS[["POS", "MUT_LEN_fixed"]]

#calculate flanking coordinates for plotting SVs
empty_regions = []
empty_regions.append(0)
empty_regions.append(chromosome_length)

CDS_SV_coords = CDS_SV_coords.reset_index()

for index, row in CDS_SV_coords.iterrows():
    empty_regions.append(CDS_SV_coords["POS"][index] -1)
    empty_regions.append(CDS_SV_coords["POS"][index] + 1)

empty_regions = list(set(empty_regions))

CDS_plot_SV_coords = pd.DataFrame()
CDS_plot_SV_coords["Coords"] = empty_regions
CDS_plot_SV_coords["MUT_LEN_fixed"] = 0
CDS_plot_SV_coords1 = pd.DataFrame()
CDS_plot_SV_coords1["Coords"] = CDS_SV_coords["POS"]
CDS_plot_SV_coords1["MUT_LEN_fixed"] = CDS_SV_coords["MUT_LEN_fixed"]
CDS_plot_SV_coords2 = pd.concat([CDS_plot_SV_coords, CDS_plot_SV_coords1], axis = 0)

In [ ]:
#save all
overlap_df1.to_csv("./result/checkpoint_fin/overlaps/overlap_df_chr1.csv", index=False)
tr_cov_df.to_csv("./result/checkpoint_fin/overlaps/tr_cov_plot/tr_cov_df_chr1.csv", index=False)
overlap_df1_CDS.to_csv("./result/checkpoint_fin/overlaps/CDS/overlap_df_chr1.csv", index=False)
tr_cov_df_CDS.to_csv("./result/checkpoint_fin/overlaps/CDS/tr_cov_plot/tr_cov_df_chr1.csv", index=False)

all_SV_coords.to_csv("./plotting/transcript_SV_coverage/checkpoint3/allSVchr1")
SV_coords.to_csv("./plotting/transcript_SV_coverage/checkpoint3/overlapSVchr1")
overlap_norm_SV_coords2.to_csv("./plotting/transcript_SV_coverage/checkpoint3/NormSVLenPerTrChr1")

CDS_plot_SV_coords2.to_csv("./plotting/transcript_SV_coverage/checkpoint3/CDSSVchr1")
CDS_norm_SV_coords2.to_csv("./plotting/transcript_SV_coverage/checkpoint3/CDS_SV_densitychr1_coords")
CDS_overlaps_test.to_csv("./plotting/transcript_SV_coverage/checkpoint3/CDS_SV_densitychr1")

tr_cov_df_CDS.reset_index().to_csv("./result/checkpoint_fin/overlaps/CDS/tr_cov_plot/tr_cov_df_chr1.csv", index=False)

In [ ]:
#whole chromosome Myriapoda plot
from matplotlib.ticker import MaxNLocator

plt.rcParams['figure.figsize'] = [40, 20]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['agg.path.chunksize'] = 10000

fig, axes = plt.subplots(nrows=3, ncols=1, gridspec_kw={'height_ratios': [10, 1, 10]},
                         sharex=True)

plt.subplots_adjust(wspace=0, hspace=0)

all_SV_coords.sort_values(by=["Coords"]).plot.line(x="Coords", y= "abs_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(0, chromosome_length), ax = axes[0], color='b', linewidth=2)

SV_coords.sort_values(by=["Coords"]).plot.line(x="Coords", y= "abs_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(0, chromosome_length), ax = axes[0], color='r', linewidth=2)

CDS_plot_SV_coords2.sort_values(by=["Coords"]).plot.line(x="Coords", y= "MUT_LEN_fixed", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(0, chromosome_length), ax = axes[0], color='g', linewidth=2)

tr_cov_df.sort_values(by=["Coords"]).plot.area(x="Coords", y= "Coverage", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(0, chromosome_length), ax = axes[1], color='r',linewidth=0.2)

overlap_norm_SV_coords2.sort_values(by=["Coords"]).plot.line(x="Coords", y= "norm_overlap_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(0, chromosome_length), ax = axes[2], color='r')

fig.suptitle("SV distribution, transcript distribution, and SV density on chromosome 1",fontsize=140)
axes[0].set_ylabel("", fontsize=120, labelpad=170)
axes[1].set_ylabel("", fontsize=120, labelpad=420)
axes[2].set_ylabel("", fontsize=120, labelpad=320)
plt.xlabel('Position on chromosome (1 x $10^7$)', fontsize=120, labelpad=110)
plt.xticks(fontsize=100)
axes[0].tick_params(axis='both', which='major', labelsize=100, pad=50, width=8, size=45)
axes[0].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[1].tick_params(axis='both', which='major', labelsize=100, pad=50, width=0, size=0)
axes[1].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[2].tick_params(axis='both', which='major', labelsize=100, pad=50, width=8, size=45)
axes[2].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[0].get_xaxis().set_visible(True)
axes[1].get_xaxis().set_visible(False)
plt.gca().xaxis.set_major_locator(MaxNLocator(prune='lower'))
axes[1].xaxis.offsetText.set_fontsize(0)
axes[1].set_yticklabels([])
# axes[1].set_yticks([0,1])
# axes[1].set_yticklabels(["Absent", "Present"])
axes[2].invert_yaxis()
axes[0].grid(color = 'green', linestyle = '--', dashes=(13, 25), linewidth = 2, alpha=0.75)
axes[2].grid(color = 'green', linestyle = '--', dashes=(13, 25), linewidth = 2, alpha=0.75)

In [ ]:
#Myriapoda plot of region around the Metallothionein gene on Dinas Powys L. rubellus chromosome 1
from matplotlib.ticker import MaxNLocator

plt.rcParams['figure.figsize'] = [40, 20]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['agg.path.chunksize'] = 10000

fig, axes = plt.subplots(nrows=3, ncols=1, gridspec_kw={'height_ratios': [10, 1, 10]},
                         sharex=True)

plt.subplots_adjust(wspace=0, hspace=0)

all_SV_coords.sort_values(by=["Coords"]).plot.line(x="Coords", y= "abs_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ylim=(0, 31), color='b', ax = axes[0], linewidth=2)

SV_coords.sort_values(by=["Coords"]).plot.line(x="Coords", y= "abs_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ylim=(0, 31), ax = axes[0], color='r', linewidth=2)

CDS_plot_SV_coords2.sort_values(by=["Coords"]).plot.line(x="Coords", y= "MUT_LEN_fixed", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ax = axes[0], color='g', linewidth=2)

tr_cov_df.sort_values(by=["Coords"]).plot.area(x="Coords", y= "Coverage", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ax = axes[1], color='r',linewidth=0.2)

tr_cov_df_CDS.sort_values(by=["Coords"]).plot.area(x="Coords", y= "Coverage", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ax = axes[1], color='g',linewidth=0.2)

overlap_norm_SV_coords2.sort_values(by=["Coords"]).plot.line(x="Coords", y= "norm_overlap_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ylim=(0, 0.15), ax = axes[2], color='r')

CDS_norm_SV_coords2.sort_values(by=["Coords"]).plot.line(
                        x="Coords", y= "norm_overlap_MUT_LEN", figsize=(200,60), grid=True,
                       xlabel="Transcript #", ylabel="Normalised total transcript length differential", 
                        legend=False, xlim=(30868202, 30876289), ax = axes[2], color='g')

fig.suptitle("Full MT2 Transcript",fontsize=140)
axes[0].set_ylabel("", fontsize=120, labelpad=170)
axes[1].set_ylabel("", fontsize=120, labelpad=420)
axes[2].set_ylabel("", fontsize=120, labelpad=320)
plt.xlabel('Position on chromosome (1 x $10^6$)', fontsize=120, labelpad=110)
plt.xticks(fontsize=100)
axes[0].tick_params(axis='both', which='major', labelsize=100, pad=50, width=8, size=45)
axes[0].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[1].tick_params(axis='both', which='major', labelsize=100, pad=50, width=0, size=0)
axes[1].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[2].tick_params(axis='both', which='major', labelsize=100, pad=50, width=8, size=45)
axes[2].tick_params(axis='both', which='minor', labelsize=100, pad=50, width=0, size=0)
axes[0].get_xaxis().set_visible(True)
axes[1].get_xaxis().set_visible(False)
plt.gca().xaxis.set_major_locator(MaxNLocator(prune='lower'))
axes[1].xaxis.offsetText.set_fontsize(0)
axes[1].set_yticklabels([])
# axes[1].set_yticks([0,1])
# axes[1].set_yticklabels(["Absent", "Present"])
axes[2].invert_yaxis()
axes[0].grid(color = 'green', linestyle = '--', dashes=(13, 25), linewidth = 2, alpha=0.75)
axes[2].grid(color = 'green', linestyle = '--', dashes=(13, 25), linewidth = 2, alpha=0.75)

In [ ]:
#Further SV statistics
all_SVs_df = df_chr1_vcf.drop_duplicates(subset=["POS"])
all_SVs_df["REF_LEN"] = all_SVs_df["REF"].apply(len)
all_SVs_df["ALT_LEN"] = all_SVs_df["ALT"].apply(len)
all_SVs_df["MUT_LEN"] = all_SVs_df["ALT_LEN"]-all_SVs_df["REF_LEN"]
all_SVs_df["abs_MUT_LEN"] = all_SVs_df["MUT_LEN"].abs()
all_SVs_df["SV_LEN"] = all_SVs_df[["REF_LEN", "ALT_LEN"]].max(axis=1)

print("Number of SVs on the chromosome: ", all_SVs_df.shape[0], 
      "\nAverage SV length: ", all_SVs_df["SV_LEN"].mean(),
      "\nSV density per chromosome: ", all_SVs_df[" "].sum()/chromosome_length,
      "\nSV density per gene coding overlap: ", 
      overlap_df1["overlap_MUT_LEN"].sum()/overlap_df1["Overlap_len"].sum())

In [ ]:
#display SV type statistics 
SV_types_df = SV_hits_test.drop_duplicates(subset=["POS"])

print("Insertions :", SV_types_df["MUT_TYPE"].value_counts()["Insertion"], 
      "\nDeletions :", SV_types_df["MUT_TYPE"].value_counts()["Deletion"], 
      "\nSubstitutions :", SV_types_df["MUT_TYPE"].value_counts()["Substitution"], 
      "\nTransitions: ", SV_types_df["SUB_TYPE"].value_counts()["Transition"], 
      "\nTransversions :", SV_types_df["SUB_TYPE"].value_counts()["Transversion"], 
      "\nMulti-substitutions", 0)

#SV_hits_test["SUB_TYPE"].value_counts()["Multi-substitution"])

In [ ]:
#coding vs non-coding based on transcript overlaps rather than raw gtf data
SV_types_df = SV_hits_test.drop_duplicates(subset=["POS"])

chromosome_length = 68399915
non_coding_length = 68399915

for index_gtf, row_gtf in overlap_df1.iterrows():
    non_coding_length = non_coding_length - (row_gtf[5]-row_gtf[4])
coding_length = chromosome_length - non_coding_length
coding_percentage = 100/chromosome_length*coding_length
SV_mapped_percentage = 100/df_chr1_vcf.shape[0]*SV_types_df.shape[0]
print("Chromosome length: ", chromosome_length, "\nCoding length: ", coding_length, 
      "\nNon coding length: ", non_coding_length, 
      "\nCoding percentage: ", "{:.2f}".format(coding_percentage), "%",
     "\nPercentage SVs mapped to coding regions: ", "{:.2f}".format(SV_mapped_percentage), "%")